In [10]:
import pandas as pd
import random
from faker import Faker

# Initialize Faker
fake = Faker()

# Set Parameters
NUM_RECORDS = 1000
PRODUCT_CATEGORIES = ["Electronics", "Clothing", "Home & Kitchen", "Books", "Toys", "Sports"]
SALES_CHANNELS = ["Online", "Retail Store", "Tele-Sales"]
LEAD_SOURCES = ["Google Ads", "Social Media", "Referrals", "Organic Search", "Email Campaign"]
CONVERSION_STATUS = ["Converted", "Not Converted", "Abandoned"]

# Function to generate fake data
def generate_sales_data(num_records):
    data = []
    for _ in range(num_records):
        product_category = random.choice(PRODUCT_CATEGORIES)
        price = round(random.uniform(10, 500), 2)  # Random price between 10 and 500
        quantity = random.randint(1, 10)  # Quantity between 1 and 10
        total_revenue = round(price * quantity, 2)
        
        record = {
            "Sale Date": fake.date_this_year(),
            "Customer ID": fake.uuid4(),
            "Customer Name": fake.name(),
            "Location": fake.city(),
            "Product Category": product_category,
            "Price": price,
            "Quantity Sold": quantity,
            "Total Revenue": total_revenue,
            "Sales Channel": random.choice(SALES_CHANNELS),
            "Lead Source": random.choice(LEAD_SOURCES),
            "Conversion Status": random.choice(CONVERSION_STATUS)
        }
        data.append(record)
    return pd.DataFrame(data)

# Generate the data
sales_data = generate_sales_data(NUM_RECORDS)

# Save to CSV
sales_data.to_csv("fake_sales_data.csv", index=False)

print("Fake sales data generated and saved as 'fake_sales_data.csv'")


Fake sales data generated and saved as 'fake_sales_data.csv'


In [17]:
import pandas as pd

# Load the dataset
file_path = "fake_sales_data.csv"  # Replace with your file path
sales_data = pd.read_csv(file_path)

# 1. Check the first few rows of the dataset
print("Initial Data Sample:")
print(sales_data.head())

# 2. Check for missing values
print("\nMissing Values Before Cleaning:")
print(sales_data.isnull().sum())

# Ensure working on a clean copy to avoid view issues
sales_data = sales_data.copy()

# 3. Fill missing values
sales_data["Customer Name"] = sales_data["Customer Name"].fillna("Unknown")
sales_data["Location"] = sales_data["Location"].fillna("Unknown Location")
sales_data["Lead Source"] = sales_data["Lead Source"].fillna("Unknown Source")
sales_data["Sales Channel"] = sales_data["Sales Channel"].fillna("Unknown Channel")

# Fill numeric missing values with the median
sales_data["Price"] = sales_data["Price"].fillna(sales_data["Price"].median())
sales_data["Quantity Sold"] = sales_data["Quantity Sold"].fillna(sales_data["Quantity Sold"].median())
sales_data["Total Revenue"] = sales_data["Total Revenue"].fillna(sales_data["Total Revenue"].median())

# 4. Remove duplicates
sales_data.drop_duplicates(inplace=True)

# 5. Correct data types
sales_data["Sale Date"] = pd.to_datetime(sales_data["Sale Date"], errors="coerce")
sales_data["Price"] = pd.to_numeric(sales_data["Price"], errors="coerce")
sales_data["Quantity Sold"] = pd.to_numeric(sales_data["Quantity Sold"], errors="coerce")
sales_data["Total Revenue"] = pd.to_numeric(sales_data["Total Revenue"], errors="coerce")

# 6. Handle invalid data (e.g., outliers)
# Remove rows where Total Revenue is 0 or negative
sales_data = sales_data[sales_data["Total Revenue"] > 0]

# Optional: Cap outliers in Price and Total Revenue columns
price_upper_limit = sales_data["Price"].quantile(0.95)
revenue_upper_limit = sales_data["Total Revenue"].quantile(0.95)

sales_data["Price"] = sales_data["Price"].clip(upper=price_upper_limit)
sales_data["Total Revenue"] = sales_data["Total Revenue"].clip(upper=revenue_upper_limit)

# 7. Standardize Text Columns
text_columns = ["Customer Name", "Location", "Product Category", "Sales Channel", "Lead Source", "Conversion Status"]
for col in text_columns:
    sales_data[col] = sales_data[col].str.strip().str.title()

# 8. Check for clean data
print("\nCleaned Data Sample:")
print(sales_data.head())

print("\nMissing Values After Cleaning:")
print(sales_data.isnull().sum())

# Save the cleaned dataset
sales_data.to_csv("cleaned_sales_data.csv", index=False)
print("\nCleaned data saved as 'cleaned_sales_data.csv'")


Initial Data Sample:
    Sale Date                           Customer ID    Customer Name  \
0  2024-07-07  19c76538-b9a8-458a-b91c-2bca721207f7     Brenda Allen   
1  2024-06-05  89818909-34c2-4c44-b3ed-9996b0fe4746       James Webb   
2  2024-12-09  2371e01e-e35d-45f2-b182-2d000b48efe2  Melissa Elliott   
3  2024-10-15  b8975e62-3f4b-4047-89c0-0a25fe9296d8   Michaela James   
4  2024-10-15  c122c23b-ae40-4c1a-a091-d5ffd0d06948     Rebecca Ball   

        Location Product Category   Price  Quantity Sold  Total Revenue  \
0      Houseview      Electronics  249.50              5        1247.50   
1      Kiddmouth      Electronics  125.28              2         250.56   
2     Nathanport   Home & Kitchen  357.26              1         357.26   
3  West Amyshire         Clothing  311.54              3         934.62   
4  Christinefort            Books  184.90              3         554.70   

  Sales Channel     Lead Source Conversion Status  
0        Online  Email Campaign         Con